# 根据要求导出`WRF-CMAQ`模拟数据
两大剖面的数据。用matlab存成三维数组，三个维度分别是水平位置、垂直高度和小时时刻，用这样的三维数组去装载各指标，包括**O3浓度、PM2.5浓度、NO2浓度、各过程贡献、各来源贡献**，例如O3浓度的三维数组是CO3（2点位,300米,4月2日13时）=75μg/m3。其中水平位置可以用1~n代表n个经纬度组合，再单独给一个对照表即可，例如2点位=（E 113.6°、N 29.1°）。

---
*@author: Evan*\
*@date: 2023-08-09*

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
from scipy.io import savemat

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append('../../src/')
from findpoint import findpoint
import ModelEvaLib as me
from namelist import *

In [2]:
lon1, lat1 = 116.6, 31.1
lon2, lat2 = 117.4, 30.1

lon3, lat3 = 116.1, 29.74
lon4, lat4 = 117.46, 30.85

lon5, lat5 = 116.8, 30.54
lon6, lat6 = 117.3, 30.5

In [3]:
chem = xr.open_dataset(cmaqfile)
met = xr.open_dataset(mcipfile)
pa = xr.open_dataset(pafile)
isam1 = xr.open_dataset(isamfile1)
isam2 = xr.open_dataset(isamfile2)

In [4]:
nlevel = 21
number = 50

lat = chem.latitude
lon = chem.longitude

O3   = chem.O3[:,:nlevel,:,:]
NO2  = chem.NO2[:,:nlevel,:,:]
PM25 = chem.PM25[:,:nlevel,:,:]
ht   = chem.HT[:,:nlevel,:,:]

# mda8   = O3.rolling(time=8).mean().resample({'time':'D'}).max(dim='time')
# NO2avg = NO2.resample({'time':'D'}).mean(dim='time')
# PMavg  = PM25.resample({'time':'D'}).mean(dim='time')
# htavg  = ht.resample({'time':'D'}).mean(dim='time')
# print(mda8.shape)

HADV = pa.HADV_O3[:,:nlevel,:,:]
ZADV = pa.ZADV_O3[:,:nlevel,:,:]
HDIF = pa.HDIF_O3[:,:nlevel,:,:]
VDIF = pa.VDIF_O3[:,:nlevel,:,:]
DDEP = pa.DDEP_O3[:,:nlevel,:,:]
CHEM = pa.CHEM_O3[:,:nlevel,:,:]

O3_AnQ = isam1.O3_AnQ[:,:nlevel,:,:]
O3_AQI = isam1.O3_AQI[:,:nlevel,:,:]
O3_AQT = isam1.O3_AQT[:,:nlevel,:,:]
O3_AQA = isam1.O3_AQA[:,:nlevel,:,:]
O3_AQP = isam1.O3_AQP[:,:nlevel,:,:]
O3_OTH = isam1.O3_OTH[:,:nlevel,:,:]
O3_BCO = isam1.O3_BCO[:,:nlevel,:,:]
O3_ICO = isam1.O3_ICO[:,:nlevel,:,:]

PM_AnQ = isam2.PM_AnQ[:,:nlevel,:,:]
PM_AQI = isam2.PM_AQI[:,:nlevel,:,:]
PM_AQT = isam2.PM_AQT[:,:nlevel,:,:]
PM_AQA = isam2.PM_AQA[:,:nlevel,:,:]
PM_AQP = isam2.PM_AQP[:,:nlevel,:,:]
PM_OTH = isam2.PM_OTH[:,:nlevel,:,:]
PM_BCO = isam2.PM_BCO[:,:nlevel,:,:]
PM_ICO = isam2.PM_ICO[:,:nlevel,:,:]

HNO3 = pa.HNO3prod[:,:nlevel,:,:]
H2O2 = pa.H2O2prod[:,:nlevel,:,:]

u = met.uwind[:,:nlevel,:,:]
v = met.vwind[:,:nlevel,:,:]
w = chem.wwind[:,:nlevel,:,:]

## Line1

In [5]:
# 生成沿两个点连线的坐标
lons1 = np.linspace(lon1,lon2,num=number)
lats1 = np.linspace(lat1,lat2,num=number)

# 选取这些点在数据集中的最接近的网格点的数据
x_index = []
y_index = []

for i in range(number): 
    out_x, out_y = findpoint(lons1[i],lats1[i],O3)
    x_index.append(out_x)
    y_index.append(out_y)
O3_sel   = O3.isel(x=x_index, y=y_index)
NO2_sel  = NO2.isel(x=x_index, y=y_index)
PM25_sel = PM25.isel(x=x_index, y=y_index)
htsel    = ht.isel(x=x_index, y=y_index)

diag_O3   = np.diagonal(O3_sel, axis1=2, axis2=3)
diag_NO2  = np.diagonal(NO2_sel, axis1=2, axis2=3)
diag_PM25 = np.diagonal(PM25_sel, axis1=2, axis2=3)
diag_ht   = np.diagonal(htsel, axis1=2, axis2=3)

diag_O3.shape

(744, 21, 50)

In [6]:
HADV_sel  = HADV.isel(x=x_index, y=y_index)
ZADV_sel  = ZADV.isel(x=x_index, y=y_index)
HDIF_sel  = HDIF.isel(x=x_index, y=y_index)
VDIF_sel  = VDIF.isel(x=x_index, y=y_index)
DDEP_sel  = DDEP.isel(x=x_index, y=y_index)
CHEM_sel  = CHEM.isel(x=x_index, y=y_index)

diag_HADV  = np.diagonal(HADV_sel, axis1=2, axis2=3)
diag_ZADV  = np.diagonal(ZADV_sel, axis1=2, axis2=3)
diag_HDIF  = np.diagonal(HDIF_sel, axis1=2, axis2=3)
diag_VDIF  = np.diagonal(VDIF_sel, axis1=2, axis2=3)
diag_DDEP  = np.diagonal(DDEP_sel, axis1=2, axis2=3)
diag_CHEM  = np.diagonal(CHEM_sel, axis1=2, axis2=3)

diag_HADV.shape

(744, 21, 50)

In [7]:
O3AnQ_sel = O3_AnQ.isel(x=x_index, y=y_index)
O3AQI_sel = O3_AQI.isel(x=x_index, y=y_index)
O3AQT_sel = O3_AQT.isel(x=x_index, y=y_index)
O3AQA_sel = O3_AQA.isel(x=x_index, y=y_index)
O3AQP_sel = O3_AQP.isel(x=x_index, y=y_index)
O3OTH_sel = O3_OTH.isel(x=x_index, y=y_index)
O3ICO_sel = O3_ICO.isel(x=x_index, y=y_index)
O3BCO_sel = O3_BCO.isel(x=x_index, y=y_index)

PMAnQ_sel = PM_AnQ.isel(x=x_index, y=y_index)
PMAQI_sel = PM_AQI.isel(x=x_index, y=y_index)
PMAQT_sel = PM_AQT.isel(x=x_index, y=y_index)
PMAQA_sel = PM_AQA.isel(x=x_index, y=y_index)
PMAQP_sel = PM_AQP.isel(x=x_index, y=y_index)
PMOTH_sel = PM_OTH.isel(x=x_index, y=y_index)
PMICO_sel = PM_ICO.isel(x=x_index, y=y_index)
PMBCO_sel = PM_BCO.isel(x=x_index, y=y_index)

diag_O3AnQ = np.diagonal(O3AnQ_sel, axis1=2, axis2=3)
diag_O3AQI = np.diagonal(O3AQI_sel, axis1=2, axis2=3)
diag_O3AQT = np.diagonal(O3AQT_sel, axis1=2, axis2=3)
diag_O3AQA = np.diagonal(O3AQA_sel, axis1=2, axis2=3)
diag_O3AQP = np.diagonal(O3AQP_sel, axis1=2, axis2=3)
diag_O3OTH = np.diagonal(O3OTH_sel, axis1=2, axis2=3)
diag_O3ICO = np.diagonal(O3ICO_sel, axis1=2, axis2=3)
diag_O3BCO = np.diagonal(O3BCO_sel, axis1=2, axis2=3)

diag_PMAnQ = np.diagonal(PMAnQ_sel, axis1=2, axis2=3)
diag_PMAQI = np.diagonal(PMAQI_sel, axis1=2, axis2=3)
diag_PMAQT = np.diagonal(PMAQT_sel, axis1=2, axis2=3)
diag_PMAQA = np.diagonal(PMAQA_sel, axis1=2, axis2=3)
diag_PMAQP = np.diagonal(PMAQP_sel, axis1=2, axis2=3)
diag_PMOTH = np.diagonal(PMOTH_sel, axis1=2, axis2=3)
diag_PMICO = np.diagonal(PMICO_sel, axis1=2, axis2=3)
diag_PMBCO = np.diagonal(PMBCO_sel, axis1=2, axis2=3)

diag_O3AnQ.shape

(744, 21, 50)

In [8]:
HNO3_sel = HNO3.isel(x=x_index, y=y_index)
H2O2_sel = H2O2.isel(x=x_index, y=y_index)

diag_HNO3 = np.diagonal(HNO3_sel, axis1=2, axis2=3)
diag_H2O2 = np.diagonal(H2O2_sel, axis1=2, axis2=3)

Sens = diag_H2O2/diag_HNO3

In [9]:
u_sel = u.isel(x=x_index, y=y_index)
v_sel = v.isel(x=x_index, y=y_index)
w_sel = w.isel(x=x_index, y=y_index)

diag_u = np.diagonal(u_sel, axis1=2, axis2=3)
diag_v = np.diagonal(v_sel, axis1=2, axis2=3)
diag_w = np.diagonal(w_sel, axis1=2, axis2=3)

In [10]:
df_line1 = pd.DataFrame(
    index=np.arange(1,51),
    data={'经度':lons1,'纬度':lats1}
)
df_line1.to_excel('D:/Download/line1.xlsx')

In [11]:
savemat(
    'D:/Download/AQ_2307_line1.mat',
    {
    'height':diag_ht,
    'O3':diag_O3,
    'NO2':diag_NO2,
    'PM25':diag_PM25,
    'HADV':diag_HADV,
    'ZADV':diag_ZADV,
    'HDIF':diag_HDIF,
    'VDIF':diag_VDIF,
    'DDEP':diag_DDEP,
    'CHEM':diag_CHEM,
    'O3_AnQ':diag_O3AnQ,
    'O3_AQI':diag_O3AQI,
    'O3_AQT':diag_O3AQT,
    'O3_AQA':diag_O3AQA,
    'O3_AQP':diag_O3AQP,
    'O3_OTH':diag_O3OTH,
    'O3_BCO':diag_O3BCO,
    'O3_ICO':diag_O3ICO,
    'PM_AnQ':diag_PMAnQ,
    'PM_AQI':diag_PMAQI,
    'PM_AQT':diag_PMAQT,
    'PM_AQA':diag_PMAQA,
    'PM_AQP':diag_PMAQP,
    'PM_OTH':diag_PMOTH,
    'PM_BCO':diag_PMBCO,
    'PM_ICO':diag_PMICO,
    'Sensitivity':Sens,
    'u':diag_u,
    'v':diag_v,
    'w':diag_w,
    }
    )